# Hot-Pow
### Investigating SnowEx Temperature Observations at Grand Mesa, Colorado, Winter 2019-2020

#### Learning objectives
* **Data Wrangling**
    * How can we pull in new external data sources and format these for use
* **Data Visualization**
    * Timeseries plotting with matplotlib and other interactive plotters (e.g., Holoviz)
* **Advanced Geospatial Visualization**
    * GIS style and multi-dimensional plotting
* **Exploring Science Questions using Thermal Data**
    * *Add more here which gets echoed under summary* 
    * How does SnowEx data (especially temperatures) compare to the ERA5 reanalysis product? 
    * How do near surface temperatures taken at snow puts compare to other thermal datasets? and more...

In [1]:
# add flowchart for going from in-situ to airborne to model data

## Grand Mesa, Colorado, Winter 2019-2020

![](/home/jovyan/hot-pow/notebooks/images/grand_mesa_snotel.png)

## Snow pit temperature data
*Add brief description....*    

In [3]:
# add map of snowpit distribution

#### Data wrangling
*Add description of where the data came from and what needed to be done to get it analysis ready.*   
*Feel free to just load the preprocessed file from disk, instead of going through all the preprocessing steps.*

#### Visualization

#### Takeaways

## Thermal IR
*Add brief description....*  

#### Data wrangling
*Add description of where the data came from and what needed to be done to get it analysis ready.*   
*Feel free to just load the preprocessed file from disk, instead of going through all the preprocessing steps.*

#### Visualization

#### Takeaways

## ERA5 vs. SNOTEL vs. SnowEx Temperature comparison
*Add brief description....*  

#### Data wrangling
*Add description of where the data came from and what needed to be done to get it analysis ready.*   
*Feel free to just load the preprocessed file from disk, instead of going through all the preprocessing steps.*

#### Visualization

#### Takeaways

## Conopy shadows and impact on snow surface temperature
*Add brief description....*  

#### Data wrangling
*Add description of where the data came from and what needed to be done to get it analysis ready.*   
*Feel free to just load the preprocessed file from disk, instead of going through all the preprocessing steps.*

#### Visualization

#### Takeaways

## Summary
*Summary and takeaways*